In [12]:
import pandas as pd

# Load user data
data_path = "../../data/raw/ml-100k/"

ratings_df = pd.read_csv(data_path + "u.data")

users_df = pd.read_csv(data_path + "u.user")

movie_columns = [
    "movie_id",
    "title",
    "release_date",
    "video_release_date",
    "IMDb_URL",
    "unknown",
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
movies_df = pd.read_csv(
    data_path + "u.item", sep="|", names=movie_columns, encoding="latin-1"
)

# Perform necessary preprocessing
# Example: One-hot encoding for categorical variables (gender, occupation)
users_df = pd.get_dummies(users_df, columns=["gender", "occupation"])

users_df["zip_code"] = pd.to_numeric(users_df["zip_code"], errors="coerce").fillna(0)
users_df["zip_code"] = users_df["zip_code"].astype(int)

print(users_df.shape)
# users_df.head()
ratings_df.head()

(943, 26)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
# Assuming ratings_df is already defined
# Merge the user demographics with their ratings
combined_df = pd.merge(ratings_df, users_df, on='user_id')

combined_df.head()

,user_id,movie_id,rating,timestamp,age,zip_code,gender_F,gender_M,occupation_administrator,occupation_artist,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,196,242,3,881250949,49,55105,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1,196,393,4,881251863,49,55105,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
2,196,381,4,881251728,49,55105,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,196,251,3,881251274,49,55105,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
4,196,655,5,881251793,49,55105,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True


In [14]:
# Prepare features and target
X = combined_df.drop(['user_id', 'movie_id', 'rating'], axis=1)
y = combined_df['rating']  # or convert this to a binary like/dislike


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
predictions = rf.predict(X_test)

# Evaluate the model
# accuracy = accuracy_score(y_test, predictions)
# Calculate RMSE
rmse = mean_squared_error(y_test, predictions, squared=False)

# Calculate MAE
mae = mean_absolute_error(y_test, predictions)

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

RMSE: 1.2187151704424897
MAE: 0.8606


In [20]:
def recommend_movies(user_profile, movies_df, rf_model, top_k=10):
    # Generate features for each movie for the given user profile
    user_features = user_profile.drop(['user_id'], axis=1)
    features_df = pd.concat([movies_df] * len(user_features), ignore_index=True)
    features_df[user_features.columns] = user_features.iloc[0]

    # Predict ratings for these movies
    predicted_ratings = rf_model.predict(features_df)

    # Get top-k movie recommendations
    top_k_movies = features_df.iloc[predicted_ratings.argsort()[-top_k:]]
    return top_k_movies['movie_id']


In [21]:
user_id = 1  # Replace with the user ID you're making recommendations for
user_profile = users_df[users_df['user_id'] == user_id]
print(user_profile)
recommended_movies = recommend_movies(user_profile, movies_df, rf, 10)
print(recommended_movies)


   user_id  age  zip_code  gender_F  gender_M  occupation_administrator  \
0        1   24     85711     False      True                     False   

   occupation_artist  occupation_doctor  occupation_educator  \
0              False              False                False   

   occupation_engineer  ...  occupation_marketing  occupation_none  \
0                False  ...                 False            False   

   occupation_other  occupation_programmer  occupation_retired  \
0             False                  False               False   

   occupation_salesman  occupation_scientist  occupation_student  \
0                False                 False               False   

   occupation_technician  occupation_writer  
0                   True              False  

[1 rows x 26 columns]


/var/folders/6j/7myxq8l12wz8s_qhqdqq6d4r0000gn/T/ipykernel_66585/451410127.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  features_df[user_features.columns] = user_features.iloc[0]
/var/folders/6j/7myxq8l12wz8s_qhqdqq6d4r0000gn/T/ipykernel_66585/451410127.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  features_df[user_features.columns] = user_features.iloc[0]
/var/folders/6j/7myxq8l12wz8s_qhqdqq6d4r0000gn/T/ipykernel_66585/451410127.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame b

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Action
- Adventure
- Animation
- Children
- Comedy
- ...
Feature names seen at fit time, yet now missing:
- timestamp
